# Production simulation of a Molten NaCl Salt with OpenMM

The settings are practically identical to `initial.ipynb`, the only differences are:

- Restart from the last snapshot of an NVE production run.
- There is no NpT or NVT re-equilibration, just continuation of the NVE run with more steps.

## Setup

In [ ]:
import mdtraj
import nglview
from bhmtf import add_nacl_forces, load_nacl
from openmm import unit
from stepup.core.api import amend
from utils import make_plots, runmd

In [ ]:
# All parameters in this cell can be changed through papermill:
# - https://papermill.readthedocs.io/
# - https://reproducible-reporting.github.io/stepup-reprep/stable/reference/stepup.reprep.api/#stepup.reprep.api.execute_papermill

seed = 42
part = 1
nstep_nve = 8000

# Time-related settings
timestep = 5 * unit.femtosecond
stride = 10

In [ ]:
# Inform StepUp of the output files it should expect, and which files are used as inputs.
prefix = f"sim{seed:04d}_part{part:02d}"
print("Random seed:", seed)
print("Prefix:", prefix)
print("Number of steps:", nstep_nve)
amend(
    inp=[
        "bhmtf.py",
        "utils.py",
        f"output/sim{seed:04d}_part00_first.pdb",
        f"output/sim{seed:04d}_part{part - 1:02d}_nve_last.chk",
    ],
    out=[
        f"output/{prefix}_nve_traj.csv",
        f"output/{prefix}_nve_traj.dcd",
        f"output/{prefix}_nve_last.pdb",
        f"output/{prefix}_nve_last.chk",
    ],
)

## Continuation in the NVE ensemble

In [ ]:
# Load the final structure of the exploration run.
system, topology, atnums, atcoords_init = load_nacl(f"output/sim{seed:04d}_part00_first.pdb")
add_nacl_forces(system, topology, do_charge=True, cutoff=1.5 * unit.nanometer)

In [ ]:
runmd(
    f"{prefix}_nve",
    system,
    topology,
    nstep=nstep_nve,
    timestep=timestep,
    stride=stride,
    load_checkpoint_from=f"output/sim{seed:04d}_part{part - 1:02d}_nve_last.chk",
)

In [ ]:
traj = mdtraj.load(f"output/{prefix}_nve_traj.dcd", top=f"output/sim{seed:04d}_part00_first.pdb")
view = nglview.show_mdtraj(traj)
view.add_unitcell()
view

In [ ]:
make_plots(f"{prefix}_nve")